<a href="https://colab.research.google.com/github/joker2017/Calculator/blob/master/29122018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
print( os.listdir('/content') )

Saving imagenet_class_names.txt to imagenet_class_names (3).txt
User uploaded file "imagenet_class_names.txt" with length 32673 bytes
['.config', 'imagenet_class_names (1).txt', 'imagenet_class_names.txt', 'inception_v3_2016_08_28', 'imagenet_class_names (3).txt', 'flower_photos', 'imagenet_class_names (2).txt', 'sample_data']


In [3]:


#Загрузите последнюю предварительную модель Inception v3: контрольная точка доступна по адресу https://github.com/tensorflow/models/tree/master/research/slim . 
#писок имен классов доступен по адресу https://goo.gl/brXRtZ , но вы должны сначала вставить «background» класс.
import os
import re 
import sys
import tarfile
import numpy as np
import tensorflow as tf
from random import sample
from six.moves import urllib
import matplotlib.image as mpimg
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import inception
from skimage.transform import rescale, resize, downscale_local_mean

MODEL_NAME = "inception_v3.ckpt"
CLASS_NAME_REGEX = re.compile(r"^n\d+\s+(.*)\s*$", re.M | re.U)
DATASET_URL = "http://download.tensorflow.org/example_images/flower_photos.tgz"
MODEL_URL = "http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz"



def download_progress(count, block_size, total_size):
    percent = count * block_size * 100 // total_size
    sys.stdout.write("\rDownloading: {}%".format(percent))
    sys.stdout.flush()
    

def download_tgz(url):
    file_name=os.path.basename(url)
    name_parts = os.path.split(file_name)
    _, name = name_parts
    name_path = name.rsplit( ".", 2 )[ 0 ]
    path = os.path.join("/content", name_path)
    tgz_path = os.path.join(path, name)
    if os.path.exists(path) and os.path.isfile(tgz_path):
      print("file and path exist ", tgz_path, "Path is:", os.listdir(path), "!")
      return path
    else:
      print(os.path.exists(path), os.path.isfile(tgz_path), path, tgz_path)
      os.makedirs(path, exist_ok=True)
      urllib.request.urlretrieve(url, tgz_path, reporthook=download_progress)
      tmp_tgz = tarfile.open(tgz_path)
      tmp_tgz.extractall(path=path)
      tmp_tgz.close()
      os.remove(tgz_path)
      print("path created:", path)
      return path
    
# Загружаем модель
MODELS_PATH = download_tgz(MODEL_URL)
# Загружаем датасет
FLOWERS_PATH = download_tgz(DATASET_URL)
 

def load_class_names():
    with open(os.path.join("/content", "imagenet_class_names.txt"), "rb") as f:
        content = f.read().decode("utf-8")
        #sys.stdout.flush()
        return CLASS_NAME_REGEX.findall(content)
      
load_class_names()    
class_names = ["background"] + load_class_names()       


True False /content/inception_v3_2016_08_28 /content/inception_v3_2016_08_28/inception_v3_2016_08_28.tar.gz
Downloading: 100%path created: /content/inception_v3_2016_08_28
True False /content/flower_photos /content/flower_photos/flower_photos.tgz
Downloading: 99%path created: /content/flower_photos


In [0]:
from tensorflow.contrib.slim.nets import inception
import tensorflow.contrib.slim as slim

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()

end_points

In [0]:
flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="InceptionV3")
flower_vars

In [0]:
#MODEL_URL
import os
name_parts = os.path.splitext(MODEL_URL)  
file, name = name_parts
print(file, name)
name_parts = os.path.split(MODEL_URL)  
file, name = name_parts
print(file, name)
print(name.rsplit( ".", 2 )[ 0 ]) 
print(os.path.dirname(MODEL_URL))

In [0]:
import os
print( os.getcwd() )
print( os.listdir('/content/flower_photos/flower_photos') )

In [0]:
class_names[:5]

In [4]:
width = 299
height = 299
channels = 3
import tensorflow as tf
tf.reset_default_graph()
n_epochs = 1000
batch_size = 50
best_loss_val = np.infty
check_interval = 500
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 





#Каждый подкаталог flower_photos каталога содержит все изображения данного класса. Давайте получим список классов:
flowers_root_path = os.path.join(FLOWERS_PATH, "flower_photos")
flower_classes = sorted([dirname for dirname in os.listdir(flowers_root_path)
                  if os.path.isdir(os.path.join(flowers_root_path, dirname))])

#Давайте получим список всех путей файлов изображений для каждого класса:

from collections import defaultdict
image_paths = defaultdict(list)
for flower_class in flower_classes:
    image_dir = os.path.join(flowers_root_path, flower_class)
    for filepath in os.listdir(image_dir):
        if filepath.endswith(".jpg"):
            image_paths[flower_class].append(os.path.join(image_dir, filepath))
            
            
#Давайте сортируем пути изображения, чтобы заставить этот ноутбук вести себя последовательно на нескольких запусках:
for paths in image_paths.values():
    paths.sort() 

    
#Для получения дополнительных функций манипуляции с изображениями, таких как вращения, проверьте документацию SciPy или эту приятную страницу .
def prepare_image(image, target_width = 299, target_height = 299, max_zoom = 0.2):
    """Zooms and crops the image randomly for data augmentation."""

    
    # Во-первых, давайте найдем самый большой ограничивающий прямоугольник с целевым соотношением размеров, которое вписывается в изображение
    height = image.shape[0]
    width = image.shape[1]
    image_ratio = width / height
    target_image_ratio = target_width / target_height
    crop_vertically = image_ratio < target_image_ratio
    crop_width = width if crop_vertically else int(height * target_image_ratio)
    crop_height = int(width / target_image_ratio) if crop_vertically else height
        
   #Теперь давайте уменьшим этот ограничивающий прямоугольник случайным фактором (разделив размеры на случайное число
   # между 1.0 и 1.0 + `max_zoom`
    resize_factor = np.random.rand() * max_zoom + 1.0
    crop_width = int(crop_width / resize_factor)
    crop_height = int(crop_height / resize_factor)
    
    # Далее мы можем выбрать случайное место на изображении этого прямоугольника.
    x0 = np.random.randint(0, width - crop_width)
    y0 = np.random.randint(0, height - crop_height)
    x1 = x0 + crop_width
    y1 = y0 + crop_height
    
    # Давайте обрезать изображение, используя случайную ограничивающую рамку, которую мы построили.
    image = image[y0:y1, x0:x1]

    # Давайте также перевернем изображение по горизонтали с вероятностью 50% :
    if np.random.rand() < 0.5:
        image = np.fliplr(image)

    # Теперь давайте изменим размер изображения до целевых размеров.
    image = resize(image, (target_width, target_height), mode='reflect')
    
    # Наконец, давайте убедимся, что цвета представлены как
    # 32-бит плавает в диапазоне от 0.0 до 1.0 (на данный момент):
    return image.astype(np.float32) / 255
  

def prepare_batch(flower_paths_and_classes, batch_size):
    batch_paths_and_classes = sample(flower_paths_and_classes, batch_size)
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_batch = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_batch = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_batch, y_batch 
  
  
def prepare_data(flower_paths_and_classes):
    batch_paths_and_classes = flower_paths_and_classes
    images = [mpimg.imread(path)[:, :, :channels] for path, labels in batch_paths_and_classes]
    prepared_images = [prepare_image(image) for image in images]
    X_data = 2 * np.stack(prepared_images) - 1 # Inception expects colors ranging from -1 to 1
    y_data = np.array([labels for path, labels in batch_paths_and_classes], dtype=np.int32)
    return X_data, y_data  
  
  
#раннняя остановка
def get_model_params():
    gvars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
    return {gvar.op.name: value for gvar, value in zip(gvars, tf.get_default_session().run(gvars))}

def restore_model_params(model_params):
    gvar_names = list(model_params.keys())
    assign_ops = {gvar_name: tf.get_default_graph().get_operation_by_name(gvar_name + "/Assign")
                  for gvar_name in gvar_names}
    init_values = {gvar_name: assign_op.inputs[1] for gvar_name, assign_op in assign_ops.items()}
    feed_dict = {init_values[gvar_name]: model_params[gvar_name] for gvar_name in gvar_names}
    tf.get_default_session().run(assign_ops, feed_dict=feed_dict)  
  
  #снова получить начальный график v3. На этот раз воспользуемся trainingзаполнителем
  
X = tf.placeholder(tf.float32, shape=[None, height, width, channels], name="X")
training = tf.placeholder_with_default(False, shape=[])
with slim.arg_scope(inception.inception_v3_arg_scope()):
    logits, end_points = inception.inception_v3(X, num_classes=1001, is_training=training)

inception_saver = tf.train.Saver()


#изучить входные данные выходного уровня:
# logits.op.inputs[0].op.inputs[0]  или end_points
#end_points["PreLogits"]

#отказаться от 2-го и 3-го измерений с помощью tf.squeeze() функции
prelogits = tf.squeeze(end_points["PreLogits"], axis=[1, 2])

#добавить окончательный полностью подключенный слой поверх этого слоя:  
n_outputs = len(flower_classes)

with tf.name_scope("new_output_layer"):
    flower_logits = tf.layers.dense(prelogits, n_outputs, name="flower_logits")
    Y_proba = tf.nn.softmax(flower_logits, name="Y_proba")
    

y = tf.placeholder(tf.int32, shape=[None])

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=flower_logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    flower_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="flower_logits") #flower_logits
    training_op = optimizer.minimize(loss, var_list=flower_vars)
    print(flower_vars)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(flower_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver() 
    

    
#Разделите свой набор данных на тренировочный набор и тестовый набор. Обучите модель тренировочному набору и оцените его на тестовом наборе.
#Во-первых, мы хотим представить классы как int, а не строки:
flower_class_ids = {flower_class: index for index, flower_class in enumerate(flower_classes)}


#Будет проще перетасовать набор данных, если мы представим его как список пар filepath / class:
flower_paths_and_classes = []
for flower_class, paths in image_paths.items():
    for path in paths:
        flower_paths_and_classes.append((path, flower_class_ids[flower_class]))
        
#Затем давайте перетасовать набор данных и разделим его на обучающий набор и тестовый набор:
test_ratio = 0.2
valid_ratio = 0.4
train_size = int(len(flower_paths_and_classes) * (1 - test_ratio - valid_ratio))
test_size = int(len(flower_paths_and_classes) * test_ratio)
valid_size = int(len(flower_paths_and_classes) * valid_ratio)                
np.random.shuffle(flower_paths_and_classes)
flower_paths_and_classes_test = flower_paths_and_classes[:test_size]
flower_paths_and_classes_valid = flower_paths_and_classes[test_size:valid_size]
flower_paths_and_classes_train = flower_paths_and_classes[valid_size:]
n_iterations_per_epoch = len(flower_paths_and_classes_train) // batch_size

X_train, y_train = prepare_data(flower_paths_and_classes_train)
X_test, y_test = prepare_data(flower_paths_and_classes_test)
X_valid, y_valid = prepare_data(flower_paths_and_classes_valid)        
n_epochs = 1000
#batch_size = 50       

best_loss_val = np.infty
check_interval = 100
checks_since_last_progress = 0
max_checks_without_progress = 20
best_model_params = None 

#step

def shuffle_batch(X, batch_size, batch_n):
#rnd_idx = np.random.permutation(len(X)) # np.random.permutation Случайно переставляйте последовательность или возвращайте заданный диапазон / len возвращает длину
  rnd_idx = [i for i in range(len(X)) if i>batch_n*batch_size and i<(batch_n*batch_size+batch_size+1)]
 
 # n_batches = len(X) // batch_size #//

  for batch_idx in np.array_split(rnd_idx, batch_size):  # np.array_split делит масим на несколько массивов
        X_batch = X[batch_idx]

  yield X_batch      

############################## ВЫБИРАЕМ ПЕРЕМЕННЫЕ ##########################
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="InceptionV3") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3
#n_batches = len(X_train) // batch_size
#batch_n = 1101
#batch_n1 = 734
batch_size = 3


h2_cache = []
h2_cache_1 = []
#h2_cache_valid_1 = []
h2_cache_valid = []
################################################################################
with tf.Session() as sess:
    init.run()
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    #for batch_n in range(len(X_train)//batch_size): 
    #  for X_batch in shuffle_batch(X_train, batch_size, batch_n):
    #        h2_cache = sess.run(prelogits, feed_dict={X: X_batch})
    #        if batch_n % 100 == 0:
    #          print(h2_cache_1)
            #h2_cache.append(h2_cache_1)
            #h2_cache = np.array(h2_cache)
           # np.extend
    #for batch_n in range(len(X_valid)//batch_size): 
      #for X_batch in shuffle_batch(X_valid, batch_size, batch_n):
           # h2_cache_valid_1 = sess.run(prelogits, feed_dict={X: X_batch})
            #h2_cache_valid.extend(h2_cache_valid_1)
            #h2_cache_valid = np.array(h2_cache_valid)
    
    
############################## СОЗДАЕМ КЭШ ###################################
   
    h2_cache = prelogits.eval(feed_dict={X: X_train})
   # h2_cache_valid = sess.run(prelogits, feed_dict={X: X_valid})
################################################################################  
    
    

[<tf.Variable 'flower_logits/kernel:0' shape=(2048, 5) dtype=float32_ref>, <tf.Variable 'flower_logits/bias:0' shape=(5,) dtype=float32_ref>]
INFO:tensorflow:Restoring parameters from /content/inception_v3_2016_08_28/inception_v3.ckpt


ResourceExhaustedError: ignored

In [51]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 7.0 GB  | Proc size: 12.0 GB
GPU RAM Free: 7933MB | Used: 3508MB | Util  31% | Total 11441MB


In [50]:
print(h2_cache)
#X_train.shape

[]


In [0]:
#shuffled_idx = np.random.permutation(len(X_train))
#np.array_split(h2_cache[6,8,1], 2)
#print(shuffled_idx)
#h2_cache_1 
#print(X_batch)
h2_cache.extend(h2_cache)
h2_cache

In [45]:
with tf.Session() as sess:
    init.run()
    h2_cache = np.array(h2_cache)
    n_batches = len(h2_cache)//batch_size
    
    inception_saver.restore(sess, os.path.join(MODELS_PATH, MODEL_NAME))
    #n_batches = 100
    n_valid_batches = 100
    n_test_batches = 15
    X_valid_batches = np.array_split(X_valid, n_valid_batches)
    y_valid_batches = np.array_split(y_valid, n_valid_batches)
    X_test_batches = np.array_split(X_test, n_test_batches)
    y_test_batches = np.array_split(y_test, n_test_batches)
    n_valid_batches = 10
    n_test_batches = 10
    for epoch in range(n_epochs):
        print("Epoch", epoch, end="")
        for iteration in range(n_iterations_per_epoch):
            print(".", end="")
    
            shuffled_idx = np.random.permutation(len(h2_cache))
            print(shuffled_idx)
            hidden2_batches = np.array_split(h2_cache[shuffled_idx], n_batches)
            y_batches = np.array_split(y_train[shuffled_idx], n_batches)
            for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
               sess.run(training_op, feed_dict={prelogits:hidden2_batch, y:y_batch})

            #X_valid_1, y_valid_1 = prepare_batch(flower_paths_and_classes_valid, batch_size=len(flower_paths_and_classes_test)//5)
            #sess.run(training_op, feed_dict={X: X_batch_1, y: y_batch_1, training: True})
            if iteration % check_interval == 0:
                
                loss_val = loss.eval(feed_dict={X: X_valid_batches, y: y_valid_batches})
                print("  Train loss_val:", loss_val)
                if loss_val < best_loss_val:
                    best_loss_val = loss_val
                    checks_since_last_progress = 0
                    best_model_params = get_model_params()
                else:
                    checks_since_last_progress += 1
       
        acc_train = accuracy.eval(feed_dict={X: X_test_batches, y: y_test_batches})
        acc_val = accuracy.eval(feed_dict={X: X_valid_batches, y: y_valid_batches})
        print("Epoch {}, train accuracy: {:.4f}%, valid. accuracy: {:.4f}%, valid. best loss: {:.6f}".format(
                  epoch, acc_train * 100, acc_val * 100, best_loss_val))
        if checks_since_last_progress > max_checks_without_progress:
            print("Early stopping!")
            break

    if best_model_params:
        restore_model_params(best_model_params)
    acc_test = accuracy.eval(feed_dict={X: X_test_batches, y: y_test_batches})
    print("Final accuracy on test set:", acc_test)
    save_path = saver.save(sess, "./my_mnist_model")

INFO:tensorflow:Restoring parameters from /content/inception_v3_2016_08_28/inception_v3.ckpt
Epoch 0.[257 610 220 349 654 728 327 559 691 293   8 450 263 499 247 515 675 312
  41 151 672 266 662 446 378 680 523 106 389 103 132 191 510 117 286 196
  92 541 709 209   1 243 470 724 565 133 238 698 375 260 514 226 721 488
  18 179 108 607 231 368 249  44 306 430 481 411 227 323 505 407 305 512
 586 720 557 337  43 346 244 519 115 283 382  20 172 277 634 379 156  29
 262 438 388 573 657 367 109 635 320 254  86 264 646  45 556 552 665 275
  83 671 181   4 119 354 615  31 484 269 311 525 560  60 708  87 385 197
 121 360 452  11   2 527 618 409  93 540 116 590 473 279  85 206 380 318
 352 340 261 643  72 148 295 253 406 658 501 188 224 661 256 271 125 321
 528 390 589 289 572 699 214 594 703  53 647 651  84 462 518  22 623 702
 729 199 424 659 153 673 482 663  40  70 359 248 136  51 339 639  48 456
 522 212 466 713 475 578 668 123 686 681  78 733 683 603  49 203 509 644
 171 310 140 170 270  4

ValueError: ignored

In [59]:
shuffled_idx


array([1234, 1490,  982, ...,  469, 1880,  488])